## Imports

In [ ]:
# !pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

In [ ]:
import yaml

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import nexussdk as nxs

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# TODO: Add image explaining the dataset

## Setup

In [ ]:
ORGANIZATION = "tutorialnexus"

In [ ]:
PROJECT = "YOUR PROJECT" # TODO: Add your project

In [ ]:
DEPLOYMENT = "https://sandbox.bluebrainnexus.io/v1"

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
context = {
  "@id": "https://context.org",
  "@context": {
    "@vocab": "https://sandbox.bluebrainnexus.io/v1/vocabs/",
    "schema": "http://schema.org/",
    "Movie": {
      "@id": "schema:Movie"
    },
    "Rating": {
      "@id": "schema:Rating"
    }
  }
}

In [ ]:
nxs.config.set_environment(DEPLOYMENT)

In [ ]:
nxs.config.set_token(TOKEN)

In [ ]:
nxs.resources.create(ORGANIZATION, PROJECT, context)

In [ ]:
config = {
    "Model": {
        "name": "RdfModel",
        "origin": "store",
        "source": "BlueBrainNexus",
        "context": {
            "iri": "https://context.org",
            "bucket": f"{ORGANIZATION}/{PROJECT}"
        }
    },
    "Store": {
        "name": "BlueBrainNexus",
        "endpoint": DEPLOYMENT,
        "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
        "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson",
    },
    "Formatters": {
        "identifier": "https://movielens.org/{}/{}"
    }
}

In [ ]:
with open("./config.yml", "w") as f:
    yaml.dump(config, f)

In [ ]:
forge = KnowledgeGraphForge("./config.yml", token=TOKEN, bucket=f"{ORGANIZATION}/{PROJECT}")

## Data

In [ ]:
!curl -s -O http://files.grouplens.org/datasets/movielens/ml-latest-small.zip && unzip -qq ml-latest-small.zip && cd ml-latest-small && ls

In [ ]:
directory = "./ml-latest-small"

In [ ]:
movies_df = pd.read_csv(f"{directory}/movies.csv")

In [ ]:
ratings_df = pd.read_csv(f"{directory}/ratings.csv", dtype={"movieId":"string"})

In [ ]:
tags_df = pd.read_csv(f"{directory}/tags.csv", dtype={"movieId":"string"})

In [ ]:
links_df = pd.read_csv(f"{directory}/links.csv")

In [ ]:
movies_links_df = pd.merge(movies_df, links_df, on='movieId')

## Resources

In [ ]:
movies_links_df["type"] = "Movie"

In [ ]:
ratings_df["type"] = "Rating"

In [ ]:
tags_df["type"] = "Tag"

In [ ]:
movies_links_df["id"] = movies_links_df["movieId"].apply(lambda x: forge.format("identifier", "movies", x))

In [ ]:
movies_links_df["genres"] = movies_links_df["genres"].apply(lambda x: x.split("|"))

In [ ]:
ratings_df["movieId.id"] = movies_links_df["movieId"].apply(lambda x: forge.format("identifier", "movies", x))

In [ ]:
tags_df["movieId.id"] = tags_df["movieId"].apply(lambda x: forge.format("identifier", "movies", x))

In [ ]:
movies_resources = forge.from_dataframe(movies_links_df, np.nan, ".")

In [ ]:
ratings_resources = forge.from_dataframe(ratings_df, np.nan, ".")

In [ ]:
tags_resources = forge.from_dataframe(tags_df, np.nan, ".")

In [ ]:
print(movies_resources[0])

In [ ]:
print(ratings_resources[0])

In [ ]:
print(tags_resources[629])

## Register

In [ ]:
forge.register(movies_resources)

In [ ]:
forge.register(ratings_resources)

In [ ]:
forge.register(tags_resources)

## Query

In [ ]:
query = """
    PREFIX vocab: <https://sandbox.bluebrainnexus.io/v1/vocabs/>
    PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
    SELECT ?id ?title
    WHERE {
        ?id a Movie ;
            nxv:deprecated false ;
            vocab:title ?title ;
            ^vocab:movieId / vocab:tag "thought-provoking" . 
    }
"""

In [ ]:
resources = forge.sparql(query, limit=100, debug=True)

In [ ]:
set(forge.as_dataframe(resources).title)

In [ ]:
movie = forge.retrieve(resources[0].id)

In [ ]:
print(movie)

In [ ]:
movie._store_metadata